In [2]:
import pandas as pd
from torchtext.data import Field, BucketIterator
import torch

In [3]:
datao = pd.read_pickle("./processed.pkl")

In [4]:
# filter out 

,SRC,TRG,keywords,ext perc
32460418,Chromatin remodelers in oligodendroglia. Oligo...,B I O O O O O O O O O O O O O O O O O O O O O ...,"[CNS development, chromatin remodelers, oligod...",1
32460167,Clinical implications of serum neurofilament i...,O O O O O O O O O O O O O O O O O O O O O O B ...,"[Biomarker, Multiple sclerosis, Neurofilament ...",4
32460145,Effect of cognitive rehabilitation on neuropsy...,O O B I O O O O O O O O O O O B I O O O O O O ...,"[Clinical trial, Cognition, Cognitive rehabili...",4
32460086,B-cell depleting therapies may affect suscepti...,B I I I I O O O O O O O O O O B I O O O B O O ...,"[B-cell depleting therapies, COVID-19, DMTs, M...",4
32459513,The dichotomous role of the gut microbiome in ...,O O O O O B I O O O O B I O O O B I O O O O O ...,"[Alzheimer’s disease, Parkinson’s disease, gut...",5
...,...,...,...,...
13106716,Multiple sclerosis; its frequency and distribu...,O O O O O O O O O O O O O O O O O O O O O O O ...,[MULTIPLE SCLEROSIS/epidemiology],0
13106715,The allergic aspects of multiple sclerosis. Fo...,O O O O O O O O O O O O O O O O O O O O O O O ...,[ALLERGY/manifestations],0
13106714,Histamine treatment of multiple sclerosis. His...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[HISTAMINE/therapeutic use, MULTIPLE SCLEROSIS...",0
14832406,Studies on acute disseminated encephalomyeliti...,O O O O B O O O O O O O O O O O O O O O O O O ...,"[CEREBROSPINAL FLUID, ENCEPHALOMYELITIS]",2
